In [168]:
from preprocess import Preprocess
import numpy as np
import pandas as pd
from torchvision import transforms
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


In [169]:
IMAGE_RESIZE = (64, 64)
TRANSFORM = transforms.Compose([
    transforms.Resize(IMAGE_RESIZE),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.mean(dim=0)), # gray
    transforms.Lambda(lambda x: x.view(-1)) # flatten
])
train_data = Preprocess.load_train_pairs(transform=TRANSFORM)
val_data = Preprocess.load_test_pairs(transform=TRANSFORM)

In [170]:
# Load PCA
with open('trained/pca/pca4.pkl', 'rb') as f:
    pca = pickle.load(f)
pca

PCA(n_components=4)

In [171]:
def apply_PCA(pair_dataset):
    X = np.empty((len(pair_dataset), pca.n_components * 2))
    y = np.empty(len(pair_dataset))
    for i, (image1, image2, label) in enumerate(pair_dataset):
        image1 = pca.transform(image1.reshape(1, -1))[0]
        image2 = pca.transform(image2.reshape(1, -1))[0]
        X[i] = np.hstack((image1, image2))
        y[i] = label
    return X, y    

In [172]:
X_train, y_train = apply_PCA(train_data)
X_val, y_val = apply_PCA(val_data)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((2200, 8), (2200,), (1000, 8), (1000,))

### Regressão logística

In [173]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': np.logspace(-5, 5, 5),
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}
grid_search = GridSearchCV(LogisticRegression(max_iter=10000), param_grid, cv=5)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)

Best parameters:  {'C': 1e-05, 'solver': 'liblinear'}
Best cross-validation score:  0.5336363636363637


In [174]:
logreg_model = LogisticRegression(**grid_search.best_params_)
logreg_model.fit(X_train, y_train)

LogisticRegression(C=1e-05, solver='liblinear')

In [175]:
print('Train accuracy:', accuracy_score(y_train, logreg_model.predict(X_train)))
print('Val accuracy:', accuracy_score(y_val, logreg_model.predict(X_val)))

Train accuracy: 0.5395454545454546
Val accuracy: 0.517


### Random Forest

In [176]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_random_search = RandomizedSearchCV(RandomForestClassifier(), param_distributions=param_dist, n_iter=10, cv=5)
rf_random_search.fit(X_train, y_train)

print("Best parameters: ", rf_random_search.best_params_)
print("Best cross-validation score: ", rf_random_search.best_score_)

Best parameters:  {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 10}
Best cross-validation score:  0.5713636363636364


In [177]:
rf_model = RandomForestClassifier(**rf_random_search.best_params_)
rf_model.fit(X_train, y_train)

print('Train accuracy:', accuracy_score(y_train, rf_model.predict(X_train)))
print('Val accuracy:', accuracy_score(y_val, rf_model.predict(X_val)))

Train accuracy: 0.9522727272727273
Val accuracy: 0.58


### Rede neural

In [178]:
import keras_tuner as kt
import keras

def build_model(hp):
    model = keras.Sequential()

    # Tune the number of layers
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(keras.layers.Dense(units=hp.Int('units_' + str(i),
                                                min_value=4,
                                                max_value=32,
                                                step=4),
                                   activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=5,
    directory='runs',
    project_name=f'nn-pca-{pca.n_components}')

tuner.search_space_summary()

tuner.search(X_train, y_train, epochs=40, validation_data=(X_val, y_val))

tuner.results_summary()

best_hps = tuner.get_best_hyperparameters(num_trials = 10)[0]

Trial 5 Complete [00h 00m 31s]
val_accuracy: 0.6016000032424926

Best val_accuracy So Far: 0.6016000032424926
Total elapsed time: 00h 02m 41s
Results summary
Results in my_dir\nn-pca-4
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 4 summary
Hyperparameters:
num_layers: 1
units_0: 28
learning_rate: 0.001
units_1: 4
units_2: 12
Score: 0.6016000032424926

Trial 0 summary
Hyperparameters:
num_layers: 3
units_0: 24
learning_rate: 0.01
units_1: 4
units_2: 4
Score: 0.5972000002861023

Trial 2 summary
Hyperparameters:
num_layers: 3
units_0: 28
learning_rate: 0.001
units_1: 4
units_2: 8
Score: 0.5890000104904175

Trial 1 summary
Hyperparameters:
num_layers: 2
units_0: 12
learning_rate: 0.0001
units_1: 8
units_2: 16
Score: 0.5336000084877014

Trial 3 summary
Hyperparameters:
num_layers: 3
units_0: 8
learning_rate: 0.0001
units_1: 28
units_2: 32
Score: 0.5309999942779541


In [179]:
best_hp = tuner.get_best_hyperparameters()[0]
nn_model = tuner.hypermodel.build(best_hp)
nn_model.fit(X_train, y_train, epochs=40, validation_data=(X_val, y_val))
nn_model.summary()

Epoch 1/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4901 - loss: 0.7461 - val_accuracy: 0.5270 - val_loss: 0.7065
Epoch 2/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5165 - loss: 0.6990 - val_accuracy: 0.5350 - val_loss: 0.6908
Epoch 3/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 924us/step - accuracy: 0.5541 - loss: 0.6827 - val_accuracy: 0.5460 - val_loss: 0.6844
Epoch 4/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 966us/step - accuracy: 0.5402 - loss: 0.6826 - val_accuracy: 0.5690 - val_loss: 0.6800
Epoch 5/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 921us/step - accuracy: 0.5723 - loss: 0.6763 - val_accuracy: 0.5760 - val_loss: 0.6785
Epoch 6/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 928us/step - accuracy: 0.5938 - loss: 0.6639 - val_accuracy: 0.5780 - val_loss: 0.6756
Epoch 7/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 926us/step - accuracy: 0.5916 - loss: 0.6660 - val_accuracy: 0.5870 - val_loss: 0.6733
Epoch 8/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 977us/step - accuracy: 0.6053 - loss: 0.6691 - val_accuracy: 0.5790

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 28)             │           252 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            29 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 845 (3.30 KB)

 Trainable params: 281 (1.10 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 564 (2.21 KB)

In [180]:
train_accuracy = nn_model.evaluate(X_train, y_train)[1]
val_accuracy = nn_model.evaluate(X_val, y_val)[1]
train_accuracy, val_accuracy


69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 460us/step - accuracy: 0.6384 - loss: 0.6360
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 794us/step - accuracy: 0.5795 - loss: 0.6731


(0.6349999904632568, 0.5870000123977661)

### SVM

In [181]:
from sklearn.svm import SVC

# Define the parameter grid for grid search
param_grid = {
    'C': np.logspace(-3, 2, base=10, num=6),
    'kernel': ['rbf', 'sigmoid'],
    'gamma': np.logspace(-3, 2, base=10, num=6)
}

# Perform grid search to find the best parameters
grid_search = GridSearchCV(SVC(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)

Best parameters:  {'C': 1.0, 'gamma': 0.1, 'kernel': 'rbf'}
Best cross-validation score:  0.5790909090909091


In [182]:
# Create an SVM model with the best parameters
svm_model = SVC(**grid_search.best_params_)
svm_model.fit(X_train, y_train)

# Evaluate the model
print('Train accuracy:', accuracy_score(y_train, svm_model.predict(X_train)))
print('Val accuracy:', accuracy_score(y_val, svm_model.predict(X_val)))

Train accuracy: 0.6722727272727272
Val accuracy: 0.608
